In [23]:
from flask import Flask ,render_template, request, redirect, url_for
from PyPDF2 import PdfReader
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import traceback
import google.generativeai as genai
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain import embeddings
from langchain.prompts import PromptTemplate
import datetime
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.retrievers import KNNRetriever
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [24]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [25]:
extracted_data = load_pdf("../data/")

incorrect startxref pointer(1)


In [26]:
extracted_data

[Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 3}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 4}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 5}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 6}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 7}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 8}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 9}),
 Document(page_content='', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 10}),
 Document

In [28]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [29]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 0


In [7]:
text_split(extracted_data=extracted_data)

[]

In [8]:
load_dotenv()
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
client = OpenAI()

In [9]:
llm=OpenAI(max_tokens=1024)
embeddings = embeddings.OpenAIEmbeddings()

In [10]:
persist_directory='db'

In [11]:
loader = PyPDFDirectoryLoader("../data")

In [12]:
data = loader.load()

incorrect startxref pointer(1)


In [13]:
context = "\n".join(str(p.page_content) for p in data)
print("The total number of words in the context:", len(context))

The total number of words in the context: 636


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
texts = text_splitter.split_text(context)

In [22]:
texts

[]

In [16]:
vectordb = Chroma.from_documents(data,embedding=embeddings,persist_directory=persist_directory)

KeyboardInterrupt: 

In [ ]:
# persiste the db to disk
vectordb.persist()

In [ ]:
vectordb = None

In [ ]:
question = "what is medicine?"

In [ ]:
prompt_template = """
        Answer the question as detailed as possible from the provided context, if the answer is not in
        provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
        Context:\n {context}?\n
        Question: \n{question}\n

        Answer:
        """
        prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"],max_tokens=1024)
        chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

In [3]:
list_of_Paths=['data/computer_science_and_Technology',"data/finance_and_banking","data/Law"]
for i in range(len(list_of_Paths)):
    print(i)

0
1
2
